#### Makefiles for GRN inference

In [1]:
%%bash
code_dir="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/methods/dictys_runs"
rm -Rf $code_dir/makefiles
mkdir $code_dir/makefiles
cd $code_dir/makefiles
dictys_helper makefile_template.sh common.mk config.mk env_none.mk static.mk


In [2]:
%%bash
code_dir="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/methods/dictys_runs"
# Update configurations, such as:
# DEVICE: pytorch device, e.g. cpu, cuda:0. If you do not have a GPU, use 'cpu' and expect LONG computing time.
# GENOME_MACS2: effective genome size for macs2. See https://deeptools.readthedocs.io/en/develop/content/feature/effectiveGenomeSize.html
# JOINT: whether dataset is joint profiling of RNA and ATAC.
# Other configurations include quality control thresholds, number of threads in each job, number of hidden confounders, etc.
# They can be obtained in the full-multiome tutorial.
# NTH*j = 128, where j is the number of jobs defined during execution
dictys_helper makefile_update.py $code_dir/makefiles/config.mk '{"ENVMODE": "none", "NTH": "4", "DEVICE": "cuda:0", "GENOME_MACS2": "hs", "JOINT": "1"}'


In [14]:
!cat /ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/methods/dictys_runs/makefiles/config.mk

cat: /ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/methods/dictys_runs/makefiles/config.mk: No such file or directory


In [15]:
%%bash
#run checks on dataset and makefiles
data="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data"
makefiles="/ocean/projects/cis240075p/asachan/bio_informatics_analysis/multiome_B_Cell_analysis/B_Cell_analysis_code/method_runs/dictys_runs/makefiles"

dictys_helper makefile_check.py --dir_data $data --dir_makefiles $makefiles

Joint profile: True
Found 35420 cells with RNA profile
Found 24036 genes with RNA profile
Found 35420 cells with ATAC profile
Found 769 motifs
Found 678 TFs
Found 461 TFs in current dataset
Missing 217 TFs in current dataset: ANDR,AP2A,AP2B,AP2C,AP2D,ARI3A,ARI5B,ATF6A,BARH1,BARH2,BC11A,BHA15,BHE22,BHE23,BHE40,BHE41,BMAL1,BRAC,BSH,COE1,COT1,COT2,CR3L1,CR3L2,ERR1,ERR2,ERR3,EVI1,GCR,HEN1,HMBX1,HME1,HME2,HNF6,HTF4,HXA1,HXA10,HXA11,HXA13,HXA2,HXA5,HXA7,HXA9,HXB1,HXB13,HXB2,HXB3,HXB4,HXB6,HXB7,HXB8,HXC10,HXC11,HXC12,HXC13,HXC6,HXC8,HXC9,HXD10,HXD11,HXD12,HXD13,HXD3,HXD4,HXD8,HXD9,ITF2,KAISO,MCR,MGAP,MLXPL,MYBA,MYBB,NDF1,NDF2,NF2L1,NF2L2,NFAC1,NFAC2,NFAC3,NFAC4,NGN2,NKX21,NKX22,NKX23,NKX25,NKX28,NKX31,NKX32,NKX61,NKX62,ONEC2,ONEC3,OZF,P53,P5F1B,P63,P73,PEBB,PHX2A,PHX2B,PIT1,PKNX1,PLAL1,PO2F1,PO2F2,PO2F3,PO3F1,PO3F2,PO3F3,PO3F4,PO4F1,PO4F2,PO4F3,PO5F1,PO6F1,PO6F2,PRD14,PRGR,RHXF1,RORG,RX,SMCA1,SMCA5,SRBP1,SRBP2,STA5A,STA5B,STF1,SUH,TF2LX,TF65,TF7L1,TF7L2,TFE2,THA,THA11,THB,TWST1,TYY1,TYY2,UBIP

#### debugging cell-matching

In [3]:
%%bash
psc_path="/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data/expression.tsv.gz"
LOCAL_PATH="/dev/shm"

# Transfer with progress and verbose output
scp -v -C $psc_path $LOCAL_PATH

Executing: cp -- /ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/data/expression.tsv.gz /dev/shm


In [7]:
import pandas as pd
# load the expression matrix
expression_mtx = pd.read_csv('/dev/shm/expression.tsv', sep='\t', index_col=0)

In [8]:
#print the number of cells and genes
expression_mtx.shape

(24036, 35420)

In [9]:
# inspect the expression matrix
expression_mtx.head()

,AAACAGCCAAACCTTG-1,AAACAGCCAAAGCTAA-1,AAACAGCCAAGCCACT-1,AAACAGCCAAGGTGCA-1,AAACAGCCAAGTTATC-1,AAACAGCCAATAGCCC-1,AAACAGCCAATTATGC-1,AAACAGCCAGTTAGCC-1,AAACAGCCATAATCCG-1,AAACAGCCATTCAGCA-1,...,TTTGTTGGTGTTGCAA-1,TTTGTTGGTTAAGGTT-1,TTTGTTGGTTAGCGTA-1,TTTGTTGGTTATCCGT-1,TTTGTTGGTTGACTTC-1,TTTGTTGGTTTACGTC-1,TTTGTTGGTTTAGTCC-1,TTTGTTGGTTTATGGG-1,TTTGTTGGTTTCCTCC-1,TTTGTTGGTTTGAGGC-1
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Infer static GRNs now